In [1]:
from greensight.utils import DIR_DATA, DIR_ROOT
import pandas as pd
import numpy as np
import json
import pandas as pd

In [2]:
# load DataFrame
s2_path = DIR_DATA / "processed_data/sentinel_two_df.hdf"

df_data = pd.read_hdf(s2_path, key="df")
df_orig = df_data.copy()
df_data

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                    B12          B3          B11          B2           B9   
2017-04-01      1020.251965  623.238776  1894.865456  311.276690  4068.776515   
2017-05-01       991.962320  635.020438  1909.530406  302.160542  4384.997329   
2017-06-01      1161.614115  724.435485  2188.810410  340.702544  4549.696552   
2017-07-01      1048.994736  677.887598  2017.795291  363.510906  4043.190219   
2017-08-01              NaN         NaN          NaN         NaN          NaN   
...                     ...         ...          ...         ...          ...   
2024-06-01      1110.330183  773.270753  2035.439397  454.728518  4378.895663   
2024-07-01      1249.925666  735.594935  2288.305522  421.786959  3948.813820   
2024-08-01      1255.271394  793.528614  2250.335482  531.902502  3609.516447   
2024-09-01      1260.179600  759.827593  2132.064292  507.951894  3709.710893   
2024-10-01      1071.607682  660.778449  1924.061259  388.586222  3361.705701   

greenbelt                                                                   \
shape                                                                        
band                B8A           B5           B8          B1           B7   
2017-04-01  4035.477838  1094.207824  3999.326764  234.805820  3732.298612   
2017-05-01  4353.982110  1125.957264  4197.780260  216.930759  4058.373258   
2017-06-01  4480.317738  1301.592959  4226.873012  252.662771  4113.497425   
2017-07-01  3993.286431  1183.826676  3856.484979  300.029123  3686.713608   
2017-08-01          NaN          NaN          NaN         NaN          NaN   
...                 ...          ...          ...         ...          ...   
2024-06-01  4335.136698  1284.787768  4233.961275  395.767116  3997.971013   
2024-07-01  3941.311717  1267.901809  3733.505384  293.769673  3622.844338   
2024-08-01  3610.905452  1275.776706  3445.482901  478.499272  3273.287014   
2024-09-01  3390.136100  1222.373488  3371.289760  387.997932  3127.621285   
2024-10-01  3312.280089  1098.341848  3245.904375  342.644046  3036.303431   

greenbelt   ...         York                                        \
shape       ...    E07000167                                         
band        ...          B11          B2           B9          B8A   
2017-04-01  ...          NaN         NaN          NaN          NaN   
2017-05-01  ...  1819.617433  343.276927  3575.736009  3544.669321   
2017-06-01  ...          NaN         NaN          NaN          NaN   
2017-07-01  ...          NaN         NaN          NaN          NaN   
2017-08-01  ...          NaN         NaN          NaN          NaN   
...         ...          ...         ...          ...          ...   
2024-06-01  ...  2143.337364  520.649828  3862.858298  3857.112255   
2024-07-01  ...  2191.752492  449.142528  3689.250452  3684.549466   
2024-08-01  ...  2026.468416  628.183908  3324.942161  3079.395057   
2024-09-01  ...  2107.762038  473.549769  2652.015413  2625.643130   
2024-10-01  ...  2009.177080  578.310517  2726.817635  2687.212224   

greenbelt                                                                   \
shape                                                                        
band                 B5           B8          B1           B7           B6   
2017-04-01          NaN          NaN         NaN          NaN          NaN   
2017-05-01  1045.571895  3509.033587  249.211902  3335.098446  2744.188250   
2017-06-01          NaN          NaN         NaN          NaN          NaN   
2017-07-01          NaN          NaN         NaN          NaN          NaN   
2017-08-01          NaN          NaN         NaN          NaN          NaN   
...                 ...          ...         ...          ...          ...   
2024-06-01  1356.551093  3699.720189  468.880768  3578.255666  2991.212010   
2024-07-01 

In [3]:
lookup_path = DIR_ROOT / "resources/id_lookup/id_lookup.json"
with open(lookup_path, "r") as in_file:
    D_lookup = json.load(in_file)

In [4]:
# create weighting dataframe

# instantiate empty df
df_weights = pd.DataFrame(np.nan, index=df_data.index, columns=df_data.columns )

# apply weighting
for code, dct in D_lookup.items():
    # print(code)
    area = dct["Area_ha"]

    greenbelt_name = D_lookup[code] ["GB_Name"]
    greenbelt_area = float(np.sum([val["Area_ha"] for val in D_lookup.values() if val["GB_Name"] == greenbelt_name]))

    df_weights.loc[:, (slice(None), code, slice(None))] = area/greenbelt_area


df_weights.columns = pd.MultiIndex.from_tuples([(i[0].replace("-", " ").replace(",", ""), i[1], i[2]) for i in df_weights.columns])
df_weights.columns.names = ["greenbelt", "shape", "band"]
df_weights.head()

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                    B12        B3       B11        B2        B9       B8A   
2017-04-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-05-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-06-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-07-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-08-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   

greenbelt                                           ...      York            \
shape                                               ... E07000167             
band              B5        B8        B1        B7  ...       B11        B2   
2017-04-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   
2017-05-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   
2017-06-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   
2017-07-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   
2017-08-01  0.345098  0.345098  0.345098  0.345098  ...  0.031073  0.031073   

greenbelt                                                               \
shape                                                                    
band              B9       B8A        B5        B8        B1        B7   
2017-04-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   
2017-05-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   
2017-06-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   
2017-07-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   
2017-08-01  0.031073  0.031073  0.031073  0.031073  0.031073  0.031073   

greenbelt                       
shape                           
band              B6        B4  
2017-04-01  0.031073  0.031073  
2017-05-01  0.031073  0.031073  
2017-06-01  0.031073  0.031073  
2017-07-01  0.031073  0.031073  
2017-08-01  0.031073  0.031073  

[5 rows x 2160 columns]

In [5]:
# check all greenbelts for each band sum to 1.0
assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
df_weights.groupby(level=["greenbelt", "band"], axis=1).sum()

/tmp/ipykernel_15261/4025412648.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
/tmp/ipykernel_15261/4025412648.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_weights.groupby(level=["greenbelt", "band"], axis=1).sum()


greenbelt  Bath and Bristol                                               ...  \
band                     B1  B11  B12   B2   B3   B4   B5   B6   B7   B8  ...   
2017-04-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2017-05-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2017-06-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2017-07-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2017-08-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
...                     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2024-06-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2024-07-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2024-08-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2024-09-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2024-10-01              1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   

greenbelt  York                                               
band        B12   B2   B3   B4   B5   B6   B7   B8  B8A   B9  
2017-04-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2017-05-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2017-06-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2017-07-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2017-08-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
2024-06-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2024-07-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2024-08-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2024-09-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2024-10-01  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  

[91 rows x 192 columns]

In [6]:
# save DataFrame
output_path = DIR_DATA / "processed_data/sentinel_two_weights.hdf"
df_weights.to_hdf(output_path, key="df", mode="w", format="table")